# Implementación Manual de Gradient Boosting para Clasificación Binaria

### Carga del dataset Pima Indian Diabetes

En esta celda, se carga el dataset "Pima Indian Diabetes" desde un archivo CSV ubicado en el directorio especificado. El dataset contiene información sobre pacientes, incluyendo características como número de embarazos, nivel de glucosa, presión arterial, grosor de la piel, nivel de insulina, índice de masa corporal (BMI), función de pedigrí de diabetes y edad. Además, incluye la variable objetivo $\text{Outcome}$, que indica si el paciente tiene diabetes ($1$) o no ($0$).

Formalmente, el dataset puede representarse como una matriz $\mathbf{X} \in \mathbb{R}^{n \times d}$, donde $n$ es el número de muestras y $d$ el número de características, y un vector objetivo $\mathbf{y} \in \{0,1\}^n$:

$$
\mathbf{X} = \begin{bmatrix}
x_{1,1} & x_{1,2} & \cdots & x_{1,d} \\
x_{2,1} & x_{2,2} & \cdots & x_{2,d} \\
\vdots & \vdots & \ddots & \vdots \\
x_{n,1} & x_{n,2} & \cdots & x_{n,d}
\end{bmatrix}, \quad
\mathbf{y} = \begin{bmatrix} y_1 \\ y_2 \\ \vdots \\ y_n \end{bmatrix}
$$

Se realiza la separación de las características ($\mathbf{X}$) y la variable objetivo ($\mathbf{y}$) para su uso en el modelo de predicción.

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

# 1. Cargar dataset Pima Indian Diabetes
df = pd.read_csv("../../../datasets/pima_indian_diabetes_dataset/cleaned_dataset.csv")  # Asegúrate de tener este archivo en tu directorio
X = df.drop(columns=["Outcome"])
y = df["Outcome"]

### División de los datos en entrenamiento y prueba

En esta celda, se realiza la división del dataset en conjuntos de entrenamiento ($\mathbf{X}_{\text{train}}, \mathbf{y}_{\text{train}}$) y prueba ($\mathbf{X}_{\text{test}}, \mathbf{y}_{\text{test}}$). Esto permite evaluar el rendimiento del modelo en datos no vistos durante el entrenamiento. La proporción de división es del 80\% para entrenamiento y 20\% para prueba, utilizando una semilla aleatoria ($\text{random\_state}=42$) para garantizar la reproducibilidad de los resultados.

Formalmente:

$$
\mathbf{X}, \mathbf{y} \longrightarrow (\mathbf{X}_{\text{train}}, \mathbf{y}_{\text{train}}), (\mathbf{X}_{\text{test}}, \mathbf{y}_{\text{test}})
$$

In [2]:
# 2. Dividir datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Parámetros del modelo Gradient Boosting manual

1. **$n_{\text{rounds}}$**: Número de rondas de entrenamiento. Representa la cantidad de iteraciones en las que se ajustan árboles de decisión para mejorar las predicciones. En este caso, su valor es $10$.

2. **$\eta$ (learning rate)**: Tasa de aprendizaje utilizada para escalar las actualizaciones de los árboles en cada iteración. Controla la contribución de cada árbol al modelo final. Su valor es $1.0$.

En resumen:

- $n_{\text{rounds}} = 10$
- $\eta = 1.0$

In [3]:
# 3. Parámetros de Gradient Boosting
n_rounds = 10
learning_rate = 1.0

### Inicialización con log-odds y configuración inicial del modelo

En esta celda, se realiza la inicialización del modelo de Gradient Boosting manual. Se calcula el valor inicial de predicción constante ($y_{\text{mean}}$) utilizando el log-odds de la proporción de la variable objetivo en el conjunto de entrenamiento ($\mathbf{y}_{\text{train}}$). Este valor representa la predicción inicial antes de ajustar cualquier árbol.

La inicialización se realiza como:

$$
y_{\text{mean}} = \log\left(\frac{\bar{p}}{1-\bar{p}}\right), \quad \bar{p} = \frac{1}{N} \sum_{i=1}^N y_i
$$

donde $\bar{p}$ es la proporción de positivos en el conjunto de entrenamiento.

Además, se inicializan las predicciones para el conjunto de entrenamiento ($\hat{y}_{\text{train}}$) con el valor constante $y_{\text{mean}}$. También se crea una lista vacía para almacenar los árboles de decisión que se ajustarán en cada iteración del modelo.


In [4]:
# 4. Inicialización con log-odds (predicción constante)
y_mean = np.log(y_train.mean() / (1 - y_train.mean()))
pred_train = np.full(y_train.shape, y_mean)
trees = []

### Entrenamiento iterativo del modelo Gradient Boosting manual

En esta celda, se realiza el entrenamiento iterativo del modelo Gradient Boosting manual. Durante cada ronda de entrenamiento:

1. Se calculan las probabilidades actuales ($p_i$) utilizando la función sigmoide sobre las predicciones acumuladas ($\hat{y}_i$):
   $$
   p_i = \sigma(\hat{y}_i) = \frac{1}{1 + e^{-\hat{y}_i}}
   $$
2. Se calcula el gradiente negativo ($g_i$), que representa los residuos entre los valores reales ($y_i$) y las probabilidades actuales ($p_i$):
   $$
   g_i = y_i - p_i
   $$
3. Se ajusta un árbol de decisión $h_t(\mathbf{x})$ con profundidad máxima de 1 utilizando el gradiente como objetivo.
4. Se actualizan las predicciones acumuladas:
   $$
   \hat{y}_i \leftarrow \hat{y}_i + \eta \cdot h_t(\mathbf{x}_i)
   $$
5. Se almacena el árbol ajustado en la lista para su uso en futuras predicciones.

Al final de cada ronda, se calcula el AUC acumulado en el conjunto de entrenamiento para evaluar el rendimiento del modelo. Este proceso se repite por el número de rondas especificado ($n_{\text{rounds}}$).


In [5]:

# 5. Entrenamiento iterativo
for t in range(n_rounds):
    p = 1 / (1 + np.exp(-pred_train))  # Probabilidad actual
    gradient = y_train - p             # Gradiente negativo (residuo)

    tree = DecisionTreeRegressor(max_depth=1, random_state=42)
    tree.fit(X_train, gradient)

    update = tree.predict(X_train)
    pred_train += learning_rate * update
    trees.append(tree)

    final_preds = 1 / (1 + np.exp(-pred_train))
    auc = roc_auc_score(y_train, final_preds)
    print(f"Ronda {t+1}: AUC acumulado (train) = {auc:.4f}")


Ronda 1: AUC acumulado (train) = 0.7554
Ronda 2: AUC acumulado (train) = 0.7861
Ronda 3: AUC acumulado (train) = 0.7861
Ronda 4: AUC acumulado (train) = 0.8382
Ronda 5: AUC acumulado (train) = 0.8412
Ronda 6: AUC acumulado (train) = 0.8607
Ronda 7: AUC acumulado (train) = 0.8642
Ronda 8: AUC acumulado (train) = 0.8666
Ronda 9: AUC acumulado (train) = 0.8709
Ronda 10: AUC acumulado (train) = 0.8765


### Predicción final en el conjunto de prueba

En esta celda, se realiza la predicción final utilizando el modelo de Gradient Boosting manual en el conjunto de prueba ($\mathbf{X}_{\text{test}}$). 

1. Se inicializan las predicciones ($\hat{y}_{\text{test}}$) con el valor constante calculado previamente ($y_{\text{mean}}$).
2. Se actualizan las predicciones acumuladas sumando el producto de la tasa de aprendizaje ($\eta$) y las predicciones de cada árbol ajustado durante el entrenamiento:
   $$
   \hat{y}_{\text{test}} \leftarrow \hat{y}_{\text{test}} + \eta \cdot h_t(\mathbf{x}_{\text{test}})
   $$
3. Se calculan las probabilidades finales aplicando la función sigmoide:
   $$
   p_{\text{final}} = \sigma(\hat{y}_{\text{test}}) = \frac{1}{1 + e^{-\hat{y}_{\text{test}}}}
   $$
4. Se determinan las clases finales utilizando un umbral de 0.5:
   $$
   \hat{y}_{\text{final}} = \begin{cases} 1 & \text{si } p_{\text{final}} \geq 0.5 \\ 0 & \text{en otro caso} \end{cases}
   $$

Estas predicciones finales se utilizarán para evaluar el rendimiento del modelo en el conjunto de prueba.


In [6]:
# 6. Predicción final en test
pred_test = np.full(y_test.shape, y_mean)
for tree in trees:
    pred_test += learning_rate * tree.predict(X_test)

final_probs = 1 / (1 + np.exp(-pred_test))  # Probabilidad
final_classes = (final_probs >= 0.5).astype(int)  # Clasificación

### Evaluación del modelo Gradient Boosting manual

En esta celda, se evalúa el rendimiento del modelo Gradient Boosting manual en el conjunto de prueba ($\mathbf{X}_{\text{test}}$) utilizando las métricas de evaluación:

1. **AUC (Area Under the Curve)**: Representa la capacidad del modelo para distinguir entre las clases. Un valor más cercano a 1 indica un mejor rendimiento. Se calcula como el área bajo la curva ROC.
2. **Accuracy**: Proporción de predicciones correctas realizadas por el modelo en el conjunto de prueba:
   $$
   \text{Accuracy} = \frac{1}{n} \sum_{i=1}^n \mathbb{I}(y_i = \hat{y}_i)
   $$
   donde $\mathbb{I}$ es la función indicadora.
3. **Matriz de confusión**: Muestra el número de verdaderos positivos (TP), verdaderos negativos (TN), falsos positivos (FP) y falsos negativos (FN), proporcionando una visión detallada del rendimiento del modelo:
   $$
   \begin{bmatrix}
   \text{TN} & \text{FP} \\
   \text{FN} & \text{TP}
   \end{bmatrix}
   $$

Los resultados obtenidos se imprimen para analizar la efectividad del modelo en la tarea de clasificación.

In [7]:
# 7. Evaluación
final_auc = roc_auc_score(y_test, final_probs)
accuracy = accuracy_score(y_test, final_classes)
cm = confusion_matrix(y_test, final_classes)

print(f"\nAUC en test (Gradient Boosting manual): {final_auc:.4f}")
print(f"Accuracy en test: {accuracy:.4f}")
print("Matriz de confusión:")
print(cm)


AUC en test (Gradient Boosting manual): 0.8333
Accuracy en test: 0.7215
Matriz de confusión:
[[46  0]
 [22 11]]
